In [5]:
import torch
import torch.nn as nn

from torchvision import datasets, transforms

## ConvNet Configuration A 

In [3]:
class VGG11_configuration_A(nn.Module):
    def __init__(self,in_channels,num_clasees=10):
        super(VGG11_configuration_A,self).__init__()
        self.in_channels = in_channels
        self.num_clasees = num_clasees 
        
        self.conv_layers = nn.Sequential(
            #(batch_size,3,32,32) --> (batch_size,64,32,32)
            nn.Conv2d(in_channels=self.in_channels,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,32,32) --> (batch_size,64,16,16) 
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,64,16,16) --> (batch_size,128,16,16)
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,16,16) --> (batch_size,128,8,8)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,128,8,8) --> (batch_size,256,8,8)
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,8,8) --> (batch_size,256,4,4)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,256,4,4) --> (batch_size,512,4,4)
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,4,4) --> (batch_size,512,4,4)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,4,4) --> (batch_size,512,2,2)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,512,2,2) --> (batch_size,512,2,2)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,2,2) --> (batch_size,512,2,2)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,2,2) --> (batch_size,512,1,1)
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,self.num_clasees)
        )
        
    def forward(self,x):
        #(batch_size,3,32,32) --> (batch_size,512,1,1)
        conv = self.conv_layers(x)
        #(batch_size,512,1,1) --> (batch_size,10)
        conv = conv.view(conv.size(0),-1)
        conv = self.linear_layers(conv)
        return conv 

In [4]:
model = VGG11_configuration_A(in_channels=3,num_clasees=1000)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")

[INFO]: 132,863,336 total parameters.


# ConvNet Configuration A-LRN

In [8]:
class VGG11_configuration_A_LRN(nn.Module):
    def __init__(self,in_channels,num_clasees=1000):
        super(VGG11_configuration_A_LRN,self).__init__()
        self.in_channels = in_channels
        self.num_clasees = num_clasees 
        self.conv_layers = nn.Sequential(
            #(batch_size,3,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=self.in_channels,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2),
            #(batch_size,64,224,224) --> (batch_size,64,112,112)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,64,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,128,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,28,28)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,256,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,14,14)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,7,7)
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,self.num_clasees)
        )
    
    def forward(self,x):
        #(batch_size,3,224,224) --> (batch_size,512,7,7)
        conv = self.conv_layers(x)
        #(batch_size,512,7,7) --> (batch_size,1000)
        conv = conv.view(conv.size(0),-1)
        conv = self.linear_layers(conv)
        return conv

In [9]:
model = VGG11_configuration_A_LRN(in_channels=3,num_clasees=1000)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")

[INFO]: 132,863,336 total parameters.


# ConvNet Configuration B

In [17]:
class VGG11_configucration_B(nn.Module):
    def __init__(self,in_channels,num_classes=1000):
        super(VGG11_configucration_B,self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layer = nn.Sequential(
            #(batch_size,3,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=self.in_channels,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,112,112)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,64,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,56,56)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,128,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,28,28)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,256,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,14,14)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,7,7)
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,self.num_classes)
        )
    def forward(self,x):
        #(batch_size,3,224,224) --> (batch_size,512,7,7)
        conv = self.conv_layer(x)
        #(batch_size,512,7,7) --> (batch_size,1000)
        conv = conv.view(conv.size(0),-1)
        conv = self.linear_layers(conv)
        return conv

In [18]:
model = VGG11_configucration_B(in_channels=3)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")

[INFO]: 133,047,848 total parameters.


# ConvNet Configuration C

In [19]:
class VGG_11_configuration_C(nn.Module):
    def __init__(self,in_channels,num_classes=1000):
        super(VGG_11_configuration_C,self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = nn.Sequential(
            #(batch_size,3,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=self.in_channels,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,112,112)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,64,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,56,56)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,128,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,28,28)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,256,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,14,14)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,7,7)
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,self.num_classes)
        )
    def forward(self,x):
        #(batch_size,3,224,224) --> (batch_size,512,7,7)
        conv = self.conv_layers(x)
        #(batch_size,512,7,7) --> (batch_size,1000)
        conv = conv.view(conv.size(0),-1)
        conv = self.linear_layers(conv)
        return conv

In [20]:
model = VGG_11_configuration_C(in_channels=3)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")

[INFO]: 133,638,952 total parameters.


# ConvNet configuration D

In [21]:
class VGG11_configucration_D(nn.Module):
    def __init__(self,in_channels,num_classes=1000):
        super(VGG11_configucration_D,self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = nn.Sequential(
            #(batch_size,3,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=self.in_channels,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,112,112)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,64,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,56,56)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,128,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,28,28)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,256,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,14,14)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,7,7)
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,self.num_classes)
        )
    def forward(self,x):
        #(batch_size,3,224,224) --> (batch_size,512,7,7)
        conv = self.conv_layers(x)
        #(batch_size,512,7,7) --> (batch_size,1000)
        conv = conv.view(conv.size(0),-1)
        conv = self.linear_layers(conv)
        return conv

In [22]:
model = VGG11_configucration_D(in_channels=3)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")

[INFO]: 138,357,544 total parameters.


# ConvNet configuration E

In [23]:
class VGG11_configucration_E(nn.Module):
    def __init__(self,in_channels,num_classes=1000):
        super(VGG11_configucration_E,self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = nn.Sequential(
            #(batch_size,3,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=self.in_channels,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,224,224)
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,64,224,224) --> (batch_size,64,112,112)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,64,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,112,112)
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,128,112,112) --> (batch_size,128,56,56)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,128,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,56,56)
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,256,56,56) --> (batch_size,256,28,28)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,256,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,28,28)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,28,28) --> (batch_size,512,14,14)
            nn.MaxPool2d(kernel_size=2,stride=2),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,14,14)
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1),
            nn.ReLU(),
            #(batch_size,512,14,14) --> (batch_size,512,7,7)
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,self.num_classes)
        )
    def forward(self,x):
        #(batch_size,3,224,224) --> (batch_size,512,7,7)
        conv = self.conv_layers(x)
        #(batch_size,512,7,7) --> (batch_size,1000)
        conv = conv.view(conv.size(0),-1)
        conv = self.linear_layers(conv)
        return conv

In [24]:
model = VGG11_configucration_E(in_channels=3)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")

[INFO]: 143,667,240 total parameters.
